In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import os.path as path
import copy
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt

from gait_analysis import AnnotationsCasia as Annotations
from gait_analysis import CasiaDataset
from gait_analysis.Config import Config
from gait_analysis import Composer


# Settings 
## Dataset

In [2]:
crop_im_size = [186,250]
c = Config()
# c.config['indexing']['grouping'] = 'person_sequence_angle'
# c.config['indexing']['selection'] = 'manual_person_sequence'
c.config['pose']['load'] = False
c.config['flow']['load'] = False
c.config['heatmaps']['load'] = False
c.config['dataset_output'] = {
#         'data': ["scenes","flows","heatmaps_LAnkle","heatmaps_RAnkle"],
        'data': ["scenes"],
        'label': "annotations"}
composer = Composer()
transformer = composer.compose()
dataset = CasiaDataset(transform=transformer)


loading configuration  one_angle
[OK]


In [3]:
c.config

{'indexing': {'grouping': 'person_sequence',
  'selection': 'manual_people_sequence',
  'people_selection': [1, 2, 3, 5, 6, 7, 8, 9, 10],
  'sequences_selection': ['bg-01',
   'bg-02',
   'cl-01',
   'cl-02',
   'nm-01',
   'nm-02']},
 'pose': {'load': False,
  'preprocess': True,
  'D': 2,
  'body_keypoints_include_list': ['LAnkle',
   'RAnkle',
   'LKnee',
   'RKnee',
   'RHip',
   'LHip']},
 'flow': {'load': False,
  'preprocess': True,
  'method': 'dense',
  'load_patches': True,
  'patch_size': 5},
 'scenes': {'load': True,
  'preprocess': False,
  'crops': False,
  'gray_scale': False,
  'load_tracked': False,
  'sequences': ['bg', 'cl', 'nm'],
  'angles': [90]},
 'heatmaps': {'load': False,
  'preprocess': True,
  'body_keypoints_include_list': ['LAnkle', 'RAnkle']},
 'dataset_output': {'data': ['scenes'], 'label': 'annotations'},
 'transformers': {'AnnotationToLabel': {'target': ['annotations']},
  'Transpose': {'swapping': (2, 0, 1), 'target': ['scenes']},
  'DimensionResize':

## neural network

In [4]:
#DESIGN PARAMETERS FOR NEURAL NETWORK
NR_LSTM_UNITS = 2 
IMAGE_INPUT_SIZE_W = 640
IMAGE_INPUT_SIZE_H = 480

IMAGE_AFTER_CONV_SIZE_W = 18
IMAGE_AFTER_CONV_SIZE_H = 13
#for 3x3 kernels, n=num_layers: len_in = 2^n*len_out + sum[i=1..n](2^i)
#CONV_LAYER_LENGTH = 5

LSTM_IO_SIZE = 18*13
LSTM_HIDDEN_SIZE = 18*13

RGB_CHANNELS = 3
TIMESTEPS = 40 # size videos
BATCH_SIZE = 1 #until now just batch_size = 1

#USE RANDOM IMAGES TO SET UP WORKING EXAMPLE
class TEST_CNN_LSTM(nn.Module):
    def __init__(self):
        super(TEST_CNN_LSTM, self).__init__()

        selected_dev = "cuda:0" if torch.cuda.is_available() else "cpu"
        self.avialable_device = torch.device(selected_dev)
        
        self.conv1 = nn.Conv2d(3,6,3) #input 640x480
        self.pool1 = nn.MaxPool2d(2,2) #input 638x478 output 319x239
        self.conv2 = nn.Conv2d(6,16,3) # input 319x239 output 317x237
        self.pool2 = nn.MaxPool2d(2,2) # input 317x237 output 158x118
        self.conv3 = nn.Conv2d(16,6,3) # input 158x118 output 156x116
        self.pool3 = nn.MaxPool2d(2,2) # input 156x116 output 78x58
        self.conv4 = nn.Conv2d(6,3,3)  # input 78x58 output 76x56
        self.pool4 = nn.MaxPool2d(2,2) # input 76x56 output 39x29
        self.conv5 = nn.Conv2d(3,1,3)  # input 39x29 output 37x27
        self.pool5 = nn.MaxPool2d(2,2) #output 37x27 output 18x13
        self.lstm1 = nn.LSTM(LSTM_IO_SIZE,
                            LSTM_HIDDEN_SIZE,
                            TIMESTEPS)# horizontal direction
        self.lstm2 = nn.LSTM(LSTM_IO_SIZE,
                            LSTM_HIDDEN_SIZE,
                            TIMESTEPS)# horizontal direction
        self.fc1 = nn.Linear(LSTM_IO_SIZE,120)
        self.fc2 = nn.Linear(120,20)
        self.fc3 = nn.Linear(20,3)
        
        #initialize hidden states of LSTM
        self.hidden = self.init_hidden()

        #print("Hidden:", _hidden)
    def init_hidden(self):
        return (torch.randn(TIMESTEPS, BATCH_SIZE, LSTM_HIDDEN_SIZE).to(self.avialable_device), 
                torch.randn(TIMESTEPS, BATCH_SIZE, LSTM_HIDDEN_SIZE).to(self.avialable_device))
    def forward(self,x):
#         print("Input list len:",len(x))
#         print("Input elemens size:", x[0].size())
#         batch_size = x[0].size()[0]

        x_arr = torch.zeros(TIMESTEPS,BATCH_SIZE,1,IMAGE_AFTER_CONV_SIZE_H,IMAGE_AFTER_CONV_SIZE_W).to(self.avialable_device)
        ## print("X arr size", x_arr.size())
        for i in range(TIMESTEPS):#parallel convolutions which are later concatenated for LSTM
            x_tmp_c1 = self.pool1(F.relu(self.conv1(x[i].float())))
            x_tmp_c2 = self.pool2(F.relu(self.conv2(x_tmp_c1)))
            x_tmp_c3 = self.pool3(F.relu(self.conv3(x_tmp_c2)))
            x_tmp_c4 = self.pool4(F.relu(self.conv4(x_tmp_c3)))
            x_tmp_c5 = self.pool5(F.relu(self.conv5(x_tmp_c4)))
            x_arr[i] = x_tmp_c5 # torch.squeeze(x_tmp_c5)
        
        x, hidden = self.lstm1(x_arr.view(TIMESTEPS,BATCH_SIZE,-1), self.hidden)
        x, hidden = self.lstm2(x, self.hidden)
        # the reshaping was taken from the documentation... and makes scense
        x = x.view(TIMESTEPS,BATCH_SIZE,LSTM_HIDDEN_SIZE) #output.view(seq_len, batch, num_dir*hidden_size)
#         x = torch.squeeze(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x) 
        x = x.permute(1,2,0)
        return x
print("Class defined")

Class defined


In [ ]:
# TODOS:
# Look for overfitting..
# use GPU: 300 samples. many epochs. learning rate maybe is too high.
# Tweak learning rate
# Increase the RNN size
# Maybe change the CNN
# Use patches. or OF
# Increase the dataset: 10 images 10peoplo+++ not too much too much variance. 
# FEEDBACK on friday..:) 

## Training-testing squeme
### hyperparameters



In [ ]:
validation_split = 0.1
lr = .0005 # reduce factos of 10 .. some epoch later.
momentum = 0.9
randomized_seed = 10
shuffle_dataset = False

### Test-Training splitting

In [ ]:
#TRAINING
test_net = TEST_CNN_LSTM()
selected_dev = "cuda:0" if torch.cuda.is_available() else "cpu"
device = torch.device(selected_dev)
print(device)
test_net.to(device)
criterion = nn.CrossEntropyLoss()
#criterion = nn.BCELoss()
optimizer = optim.SGD(test_net.parameters(), lr=lr, momentum=momentum)

dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(randomized_seed)
    np.random.shuffle(indices)
train_indices, test_indices = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, sampler=test_sampler)

# 

cuda:0


In [ ]:
# test_net.parameters
# pytorch_total_params = sum(p.numel() for p in test_net.parameters())
# pytorch_total_params


In [ ]:
### define testing function
def test_all_preds(model):
    n_batches_test = len(test_loader)

    #Time for printing
    testing_start_time = time.time()

    print('Start testing...')
    correct = 0 
    total = 0
    with torch.no_grad():
        for i, batch in enumerate(train_loader):
            inputs, labels = batch
            
            scenes = [s.to(device) for s in inputs['scenes']]
            labels = labels.to(device)
            if not labels.size()[0] == BATCH_SIZE:
                # skip uncompleted batch size NN is fixed to BATCHSIZE
                continue
            outputs = model(scenes)
#             print("Out:", len(outputs), outputs.size())
#             print("Labels:", len(labels), labels.size())
            _, predicted = torch.max(outputs.data, 1)
#             print('predicted:',len(predicted),predicted.size())
            n_errors = torch.nonzero(torch.abs(labels - predicted)).size(0)
            total += predicted.numel()
            # print('predicted',predicted)
            correct += predicted.numel() - n_errors
            # print('labels',labels)
    print('Accuracy {:.2f}%'.format(100*correct/total))
    print('...testing finished')

### training over all predictions

In [ ]:
n_batches = len(train_loader)

#Time for printing
training_start_time = time.time()
EPOCHS = 1000
train_loss_hist = np.zeros(EPOCHS)

print('Start training...')
for epoch in range(EPOCHS): 
    running_loss = 0.0
    print_every = n_batches // 10
    start_time = time.time()
    total_train_loss = 0
    
    print("Epoch:", epoch)
    running_loss = 0.0
    for i, batch in enumerate(train_loader):
        
        inputs, labels = batch
        scenes = [s.to(device) for s in inputs['scenes']]
        labels = labels.view(BATCH_SIZE,1,-1).to(device)
        if not labels.size()[0] == BATCH_SIZE:
            # skip uncompleted batch size NN is fixed to BATCHSIZE
            continue
        optimizer.zero_grad() 
        outputs = test_net(scenes)
        outputs = outputs.view(BATCH_SIZE,3,1,-1)
#         print("Out:", len(outputs), outputs.size())
#         print("Labels:", len(labels), labels.size())
        loss = criterion(outputs,labels)
        loss.backward() 
        optimizer.step()
        
        #Print statistics
        # print(loss.data.item())
        running_loss += loss.data.item()
        total_train_loss += loss.data.item()
        
        #Print every 10th batch of an epoch
        if (i + 1) % (print_every + 1) == 0:
            print("Epoch {}, {:d}% \t train_loss(mean): {:.2f} took: {:.2f}s".format(
                    epoch+1, int(100 * (i+1) / n_batches), running_loss/print_every, time.time() - start_time))
            #Reset running loss and time
            running_loss = 0.0
            start_time = time.time()
    # test after each epoch
    train_loss_hist[epoch] = total_train_loss
    test_all_preds(test_net)
    print('total training loss for epoch {}: {:.6f}'.format(epoch+1, total_train_loss))

print('...Training finished')

Start training...
Epoch: 0


/home/ron/anaconda3/envs/gait_37/lib/python3.7/site-packages/pandas/core/computation/expressions.py:180: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


Epoch 1, 11% 	 train_loss(mean): 1.38 took: 4.10s
Epoch 1, 22% 	 train_loss(mean): 1.40 took: 4.06s
Epoch 1, 33% 	 train_loss(mean): 1.38 took: 3.89s
Epoch 1, 44% 	 train_loss(mean): 1.36 took: 3.94s
Epoch 1, 55% 	 train_loss(mean): 1.37 took: 3.92s
Epoch 1, 66% 	 train_loss(mean): 1.40 took: 3.90s
Epoch 1, 77% 	 train_loss(mean): 1.41 took: 3.73s
Epoch 1, 88% 	 train_loss(mean): 1.40 took: 3.86s
Epoch 1, 100% 	 train_loss(mean): 1.38 took: 3.64s
Start testing...
Accuracy 34.11%
...testing finished
total training loss for epoch 1: 49.929591
Epoch: 1
Epoch 2, 11% 	 train_loss(mean): 1.38 took: 3.63s
Epoch 2, 22% 	 train_loss(mean): 1.38 took: 3.75s
Epoch 2, 33% 	 train_loss(mean): 1.39 took: 3.93s
Epoch 2, 44% 	 train_loss(mean): 1.37 took: 3.96s
Epoch 2, 55% 	 train_loss(mean): 1.39 took: 3.68s
Epoch 2, 66% 	 train_loss(mean): 1.39 took: 3.78s
Epoch 2, 77% 	 train_loss(mean): 1.38 took: 3.80s
Epoch 2, 88% 	 train_loss(mean): 1.37 took: 3.87s
Epoch 2, 100% 	 train_loss(mean): 1.41 took:

Epoch 15, 88% 	 train_loss(mean): 1.37 took: 3.77s
Epoch 15, 100% 	 train_loss(mean): 1.37 took: 3.67s
Start testing...
Accuracy 34.78%
...testing finished
total training loss for epoch 15: 49.327895
Epoch: 15
Epoch 16, 11% 	 train_loss(mean): 1.37 took: 3.86s
Epoch 16, 22% 	 train_loss(mean): 1.37 took: 3.81s
Epoch 16, 33% 	 train_loss(mean): 1.37 took: 3.50s
Epoch 16, 44% 	 train_loss(mean): 1.37 took: 3.55s
Epoch 16, 55% 	 train_loss(mean): 1.37 took: 3.74s
Epoch 16, 66% 	 train_loss(mean): 1.36 took: 3.37s
Epoch 16, 77% 	 train_loss(mean): 1.38 took: 3.84s
Epoch 16, 88% 	 train_loss(mean): 1.37 took: 3.64s
Epoch 16, 100% 	 train_loss(mean): 1.37 took: 3.55s
Start testing...
Accuracy 34.78%
...testing finished
total training loss for epoch 16: 49.317912
Epoch: 16
Epoch 17, 11% 	 train_loss(mean): 1.37 took: 3.66s
Epoch 17, 22% 	 train_loss(mean): 1.38 took: 4.16s
Epoch 17, 33% 	 train_loss(mean): 1.37 took: 3.62s
Epoch 17, 44% 	 train_loss(mean): 1.37 took: 3.55s
Epoch 17, 55% 	 tra

Epoch 30, 22% 	 train_loss(mean): 1.37 took: 3.84s
Epoch 30, 33% 	 train_loss(mean): 1.37 took: 4.02s
Epoch 30, 44% 	 train_loss(mean): 1.37 took: 3.62s
Epoch 30, 55% 	 train_loss(mean): 1.37 took: 3.98s
Epoch 30, 66% 	 train_loss(mean): 1.36 took: 3.63s
Epoch 30, 77% 	 train_loss(mean): 1.37 took: 3.88s
Epoch 30, 88% 	 train_loss(mean): 1.36 took: 3.95s
Epoch 30, 100% 	 train_loss(mean): 1.37 took: 3.89s
Start testing...
Accuracy 35.56%
...testing finished
total training loss for epoch 30: 49.258811
Epoch: 30
Epoch 31, 11% 	 train_loss(mean): 1.37 took: 4.10s
Epoch 31, 22% 	 train_loss(mean): 1.37 took: 3.78s
Epoch 31, 33% 	 train_loss(mean): 1.37 took: 3.88s
Epoch 31, 44% 	 train_loss(mean): 1.37 took: 3.52s
Epoch 31, 55% 	 train_loss(mean): 1.37 took: 3.65s
Epoch 31, 66% 	 train_loss(mean): 1.37 took: 3.79s
Epoch 31, 77% 	 train_loss(mean): 1.37 took: 3.82s
Epoch 31, 88% 	 train_loss(mean): 1.37 took: 3.96s
Epoch 31, 100% 	 train_loss(mean): 1.37 took: 3.72s
Start testing...
Accurac

...

Accuracy 35.56%
...testing finished
total training loss for epoch 130: 48.923129
Epoch: 130
Epoch 131, 11% 	 train_loss(mean): 1.34 took: 4.28s
Epoch 131, 22% 	 train_loss(mean): 1.37 took: 4.48s
Epoch 131, 33% 	 train_loss(mean): 1.37 took: 3.90s
Epoch 131, 44% 	 train_loss(mean): 1.36 took: 3.89s
Epoch 131, 55% 	 train_loss(mean): 1.34 took: 3.75s
Epoch 131, 66% 	 train_loss(mean): 1.33 took: 3.79s
Epoch 131, 77% 	 train_loss(mean): 1.39 took: 3.90s
Epoch 131, 88% 	 train_loss(mean): 1.37 took: 3.79s
Epoch 131, 100% 	 train_loss(mean): 1.36 took: 3.74s
Start testing...
Accuracy 36.83%
...testing finished
total training loss for epoch 131: 48.918697
Epoch: 131
Epoch 132, 11% 	 train_loss(mean): 1.35 took: 4.48s
Epoch 132, 22% 	 train_loss(mean): 1.35 took: 4.30s
Epoch 132, 33% 	 train_loss(mean): 1.37 took: 3.86s
Epoch 132, 44% 	 train_loss(mean): 1.37 took: 3.84s
Epoch 132, 55% 	 train_loss(mean): 1.37 took: 3.60s
Epoch 132, 66% 	 train_loss(mean): 1.38 took: 3.79s
Epoch 132, 77% 	 t

Epoch 145, 22% 	 train_loss(mean): 1.34 took: 4.06s
Epoch 145, 33% 	 train_loss(mean): 1.36 took: 3.66s
Epoch 145, 44% 	 train_loss(mean): 1.36 took: 3.62s
Epoch 145, 55% 	 train_loss(mean): 1.34 took: 3.70s
Epoch 145, 66% 	 train_loss(mean): 1.34 took: 3.83s
Epoch 145, 77% 	 train_loss(mean): 1.35 took: 3.72s
Epoch 145, 88% 	 train_loss(mean): 1.38 took: 3.81s
Epoch 145, 100% 	 train_loss(mean): 1.39 took: 3.98s
Start testing...
Accuracy 37.94%
...testing finished
total training loss for epoch 145: 48.864864
Epoch: 145
Epoch 146, 11% 	 train_loss(mean): 1.35 took: 3.59s
Epoch 146, 22% 	 train_loss(mean): 1.34 took: 3.69s
Epoch 146, 33% 	 train_loss(mean): 1.35 took: 3.73s
Epoch 146, 44% 	 train_loss(mean): 1.34 took: 3.91s
Epoch 146, 55% 	 train_loss(mean): 1.40 took: 3.94s
Epoch 146, 66% 	 train_loss(mean): 1.37 took: 3.97s
Epoch 146, 77% 	 train_loss(mean): 1.35 took: 3.66s
Epoch 146, 88% 	 train_loss(mean): 1.36 took: 3.98s
Epoch 146, 100% 	 train_loss(mean): 1.36 took: 3.95s
Start

Epoch 159, 44% 	 train_loss(mean): 1.38 took: 5.25s
Epoch 159, 55% 	 train_loss(mean): 1.36 took: 3.95s
Epoch 159, 66% 	 train_loss(mean): 1.35 took: 4.09s
Epoch 159, 77% 	 train_loss(mean): 1.34 took: 4.07s
Epoch 159, 88% 	 train_loss(mean): 1.35 took: 3.96s
Epoch 159, 100% 	 train_loss(mean): 1.35 took: 3.70s
Start testing...
Accuracy 37.89%
...testing finished
total training loss for epoch 159: 48.814730
Epoch: 159
Epoch 160, 11% 	 train_loss(mean): 1.34 took: 3.81s
Epoch 160, 22% 	 train_loss(mean): 1.35 took: 3.85s
Epoch 160, 33% 	 train_loss(mean): 1.37 took: 4.64s
Epoch 160, 44% 	 train_loss(mean): 1.36 took: 4.27s
Epoch 160, 55% 	 train_loss(mean): 1.36 took: 3.98s
Epoch 160, 66% 	 train_loss(mean): 1.36 took: 3.58s
Epoch 160, 77% 	 train_loss(mean): 1.34 took: 4.05s
Epoch 160, 88% 	 train_loss(mean): 1.36 took: 3.87s
Epoch 160, 100% 	 train_loss(mean): 1.35 took: 4.20s
Start testing...
Accuracy 37.89%
...testing finished
total training loss for epoch 160: 48.810930
Epoch: 160


Epoch 173, 66% 	 train_loss(mean): 1.38 took: 4.00s
Epoch 173, 77% 	 train_loss(mean): 1.38 took: 3.82s
Epoch 173, 88% 	 train_loss(mean): 1.35 took: 3.88s
Epoch 173, 100% 	 train_loss(mean): 1.37 took: 3.96s
Start testing...
Accuracy 37.89%
...testing finished
total training loss for epoch 173: 48.774005
Epoch: 173
Epoch 174, 11% 	 train_loss(mean): 1.36 took: 3.90s
Epoch 174, 22% 	 train_loss(mean): 1.38 took: 3.95s
Epoch 174, 33% 	 train_loss(mean): 1.32 took: 3.69s
Epoch 174, 44% 	 train_loss(mean): 1.35 took: 3.85s
Epoch 174, 55% 	 train_loss(mean): 1.39 took: 3.92s
Epoch 174, 66% 	 train_loss(mean): 1.32 took: 3.62s
Epoch 174, 77% 	 train_loss(mean): 1.38 took: 4.06s
Epoch 174, 88% 	 train_loss(mean): 1.36 took: 3.88s
Epoch 174, 100% 	 train_loss(mean): 1.33 took: 4.08s
Start testing...
Accuracy 37.89%
...testing finished
total training loss for epoch 174: 48.769276
Epoch: 174
Epoch 175, 11% 	 train_loss(mean): 1.36 took: 4.04s
Epoch 175, 22% 	 train_loss(mean): 1.36 took: 3.80s


Epoch 187, 88% 	 train_loss(mean): 1.33 took: 3.80s
Epoch 187, 100% 	 train_loss(mean): 1.32 took: 3.96s
Start testing...
Accuracy 37.89%
...testing finished
total training loss for epoch 187: 48.739102
Epoch: 187
Epoch 188, 11% 	 train_loss(mean): 1.37 took: 5.15s
Epoch 188, 22% 	 train_loss(mean): 1.36 took: 4.06s
Epoch 188, 33% 	 train_loss(mean): 1.36 took: 3.84s
Epoch 188, 44% 	 train_loss(mean): 1.35 took: 3.98s
Epoch 188, 55% 	 train_loss(mean): 1.36 took: 3.72s
Epoch 188, 66% 	 train_loss(mean): 1.35 took: 3.79s
Epoch 188, 77% 	 train_loss(mean): 1.32 took: 3.85s
Epoch 188, 88% 	 train_loss(mean): 1.34 took: 3.84s
Epoch 188, 100% 	 train_loss(mean): 1.36 took: 3.89s
Start testing...
Accuracy 37.89%
...testing finished
total training loss for epoch 188: 48.735922
Epoch: 188
Epoch 189, 11% 	 train_loss(mean): 1.36 took: 4.55s
Epoch 189, 22% 	 train_loss(mean): 1.33 took: 3.70s
Epoch 189, 33% 	 train_loss(mean): 1.34 took: 4.17s
Epoch 189, 44% 	 train_loss(mean): 1.36 took: 3.71s


In [1]:
plt.plot(train_loss_hist)
plt.title('train loss history')
plt.xlabel('epoch number')
plt.ylabel('train loss for all epoch')

NameError: name 'plt' is not defined

### training only last prediction

In [ ]:
### define testing function
def test_last_pred(model):
    n_batches_test = len(test_loader)

    #Time for printing
    testing_start_time = time.time()

    print('Start testing...')
    correct = 0 
    total = 0
    with torch.no_grad():
        for i, batch in enumerate(train_loader):
            inputs, labels = batch
            
            scenes = [s.to(device) for s in inputs['scenes']]
            labels = labels[:,-1].to(device)
            if not labels.size()[0] == BATCH_SIZE:
                # skip uncompleted batch size NN is fixed to BATCHSIZE
                continue
            outputs = model(scenes)
            outputs = outputs[:,:,-1]
#             print("Out:", len(outputs), outputs.size())
#             print("Labels:", len(labels), labels.size())
            _, predicted = torch.max(outputs.data, 1)
#             print('predicted:',len(predicted),predicted.size())
            n_errors = torch.nonzero(torch.abs(labels- predicted)).size(0)
            total += predicted.numel()
            # print('predicted',predicted)
            correct += predicted.numel() - n_errors
            # print('labels',labels)
    print('Accuracy {:.2f}%'.format(100*correct/total))
    print('...testing finished')

In [ ]:
n_batches = len(train_loader)
# change the optimizer lerning rate:
EPOCHS = 500
optimizer = optim.SGD(test_net.parameters(), lr=0.00001, momentum=momentum)


#Time for printing
training_start_time = time.time()
    
print('Start training...')
train_loss_hist = np.zeros(EPOCHS)
for epoch in range(EPOCHS): 
    running_loss = 0.0
    print_every = n_batches // 10
    start_time = time.time()
    total_train_loss = 0
    
    print("Epoch:", epoch)
    running_loss = 0.0
    for i, batch in enumerate(train_loader):
        
        inputs, labels = batch
        scenes = [s.to(device) for s in inputs['scenes']]
        labels = labels[:,-1].to(device)
        if not labels.size()[0] == BATCH_SIZE:
            # skip uncompleted batch size NN is fixed to BATCHSIZE
            continue
        optimizer.zero_grad() 
        outputs = test_net(scenes)
        outputs = outputs[:,:,-1]
#         print("Out:", len(outputs), outputs.size(),outputs)
#         print("Labels:", len(labels), labels.size(),labels)
        loss = criterion(outputs,labels )
        loss.backward() 
        optimizer.step()
        
        #Print statistics
#         print('loss.data.item()', loss.data.item())
        running_loss += loss.data.item()
        total_train_loss += loss.data.item()
        
        #Print every 10th batch of an epoch
        if (i + 1) % (print_every + 1) == 0:
            print("Epoch {}, {:d}% \t train_loss(mean): {:.2f} took: {:.2f}s".format(
                    epoch+1, int(100 * (i+1) / n_batches), running_loss/print_every, time.time() - start_time))
            #Reset running loss and time
            running_loss = 0.0
            start_time = time.time()
    # test after each epoch
    train_loss_hist[epoch] = total_train_loss
    test_last_pred(test_net)
    print('total training loss for epoch {}: {:.6f}'.format(epoch+1, total_train_loss))

print('...Training finished')

In [ ]:
plt.plot(train_loss_hist)
plt.title('train loss history')
plt.xlabel('epoch number')
plt.ylabel('train loss for all epoch')

In [ ]:
# for i  in range(len(dataset)):
#     inputs,labels =  dataset[i]
#     if any(x==3 for x in labels):
#         print('i',i)
#         print(dataset.dataset_items[i])
#         print('labels',labels)

In [ ]:
# dataset.dataset_items[93]

In [5]:

m = nn.LogSoftmax()
loss = nn.NLLLoss()
# input is of size N x C = 3 x 5
input = torch.randn(3, 5, requires_grad=True)
# each element in target has to have 0 <= value < C
target = torch.tensor([1, 0, 4])
output = loss(m(input), target)
print('simple N x C: ', output)

# 2D loss example (used, for example, with image inputs)
N, C = 5, 4
loss = nn.NLLLoss()
# input is of size N x C x height x width
data = torch.randn(N, 16, 10, 10)
conv = nn.Conv2d(16, C, (3, 3))
m = nn.LogSoftmax()
# each element in target has to have 0 <= value < C
target = torch.empty(N, 8, 8, dtype=torch.long).random_(0, C)
output = loss(m(conv(data)), target)
output.backward()

simple N x C:  tensor(2.0539, grad_fn=<NllLossBackward>)


/Users/ari/anaconda3/envs/gait_36/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  
/Users/ari/anaconda3/envs/gait_36/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [17]:
loss = nn.CrossEntropyLoss()
pred = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(pred, target)
print('input:  ', pred)
print('target: ', target)
print('output: ', output)

input:   tensor([[-0.1556, -0.0469,  0.2248, -0.9187, -0.2758],
        [-0.9517, -0.2217, -0.3189, -1.9831,  0.0211],
        [ 2.0368,  2.0079, -0.0788, -0.1489, -0.0447]], requires_grad=True)
target:  tensor([1, 4, 0])
output:  tensor(1.1446, grad_fn=<NllLossBackward>)


In [39]:
target = torch.tensor([1,2,0,1,2,0,1,2,0,1],dtype=torch.long)
pred = torch.tensor([[0,100,0],
                        [0,0,10],
                        [10,0,0],
                        [0,10,0],
                        [0,0,10],
                        [10,0,0],
                        [0,10,0],
                        [0,0,10],
                        [10,0,0],
                        [0,10,0]],dtype=torch.float)
output = loss(pred,target)
print('input:  ', pred)
print('target: ', target)
print('output: ', output)
print('===> loss = ',output)

input:   tensor([[  0., 100.,   0.],
        [  0.,   0.,  10.],
        [ 10.,   0.,   0.],
        [  0.,  10.,   0.],
        [  0.,   0.,  10.],
        [ 10.,   0.,   0.],
        [  0.,  10.,   0.],
        [  0.,   0.,  10.],
        [ 10.,   0.,   0.],
        [  0.,  10.,   0.]])
target:  tensor([1, 2, 0, 1, 2, 0, 1, 2, 0, 1])
output:  tensor(8.1539e-05)
===> loss =  tensor(8.1539e-05)


In [50]:
target = torch.tensor([[1,2,0,1,2,0,1,2,0,1],
                       [1,2,0,1,2,0,1,2,0,1],
                       [1,2,0,1,2,0,1,2,0,1],
                       [1,2,0,1,2,0,1,2,0,1],
                       [1,2,0,1,2,0,1,2,0,1]],dtype=torch.long)
pred = torch.tensor([[[0,100,0],
                        [0,0,10],
                        [10,0,0],
                        [0,10,0],
                        [0,0,10],
                        [10,0,0],
                        [0,10,0],
                        [0,0,10],
                        [10,0,0],
                        [0,10,0]],
                    [[0,100,0],
                        [0,0,10],
                        [10,0,0],
                        [0,10,0],
                        [0,0,10],
                        [10,0,0],
                        [0,10,0],
                        [0,0,10],
                        [10,0,0],
                        [0,10,0]],
                    [[0,100,0],
                        [0,0,10],
                        [10,0,0],
                        [0,10,0],
                        [0,0,10],
                        [10,0,0],
                        [0,10,0],
                        [0,0,10],
                        [10,0,0],
                        [0,10,0]],
                    [[0,100,0],
                        [0,0,10],
                        [10,0,0],
                        [0,10,0],
                        [0,0,10],
                        [10,0,0],
                        [0,10,0],
                        [0,0,10],
                        [10,0,0],
                        [0,10,0]],
                    [[0,100,0],
                        [0,0,10],
                        [10,0,0],
                        [0,10,0],
                        [0,0,10],
                        [10,0,0],
                        [0,10,0],
                        [0,0,10],
                        [10,0,0],
                        [0,10,0]]],dtype=torch.float).permute(0,2,1)
print('input to the loss funciton: ', pred)
print('labels to the loss function: ', target)

print('BEFORE')
print('input:  ', pred.size())
print('target: ', target.size())

pred = pred.unsqueeze(-1)
pred = pred.permute(0, 2, 1, 3).squeeze(3).squeeze(0)
target = target.unsqueeze(-1)
target = target.squeeze(2).squeeze(0)
# pred = pred.view(5, 3, 1, -1)
# target = target.view(5,1,-1)

print('AFTER')
print('input:  ', pred.size())
print('target: ', target.size())

output = loss(pred,target)
print('output: ', output)
print('===> loss = ',output)

input to the loss funciton:  tensor([[[  0.,   0.,  10.,   0.,   0.,  10.,   0.,   0.,  10.,   0.],
         [100.,   0.,   0.,  10.,   0.,   0.,  10.,   0.,   0.,  10.],
         [  0.,  10.,   0.,   0.,  10.,   0.,   0.,  10.,   0.,   0.]],

        [[  0.,   0.,  10.,   0.,   0.,  10.,   0.,   0.,  10.,   0.],
         [100.,   0.,   0.,  10.,   0.,   0.,  10.,   0.,   0.,  10.],
         [  0.,  10.,   0.,   0.,  10.,   0.,   0.,  10.,   0.,   0.]],

        [[  0.,   0.,  10.,   0.,   0.,  10.,   0.,   0.,  10.,   0.],
         [100.,   0.,   0.,  10.,   0.,   0.,  10.,   0.,   0.,  10.],
         [  0.,  10.,   0.,   0.,  10.,   0.,   0.,  10.,   0.,   0.]],

        [[  0.,   0.,  10.,   0.,   0.,  10.,   0.,   0.,  10.,   0.],
         [100.,   0.,   0.,  10.,   0.,   0.,  10.,   0.,   0.,  10.],
         [  0.,  10.,   0.,   0.,  10.,   0.,   0.,  10.,   0.,   0.]],

        [[  0.,   0.,  10.,   0.,   0.,  10.,   0.,   0.,  10.,   0.],
         [100.,   0.,   0.,  10.,   0., 

ValueError: Expected target size (5, 3), got torch.Size([5, 10])